# Jupyter Rich Display

At present, the mimetype (also called a _media type_) claimed by this extension is `application/x-drawio`.

In [7]:
MIME_TYPE = "application/x-drawio"

In [8]:
import lxml.etree as ET, pathlib as P, IPython.display as D, ipywidgets as W

In [9]:
a_xml = P.Path("A.dio").read_text()

In [10]:
class RichDiagram:
    _STRIP_KEYS = ["host", "modified", "agent"]
    
    def __init__(self, data: str=a_xml, **metadata):
        if "height" not in metadata:
            metadata["height"] = "600px"
        self._metadata = metadata
        self._data = data
    
    def __repr__(self):
        data, metadata = self._clean()
        return data

    def _repr_mimebundle_(self, **kwargs):
        data, metadata = self._clean()
        return {
            MIME_TYPE: data,
            "text/markdown": f"```md\n{data}\n```",
            "text/plain": data
        }, {MIME_TYPE: metadata}
    
    def _clean(self):
        node = ET.fromstring(self._data)
        node.attrib #.getchildren()[0]
        [node.attrib.pop(key, None) for key in self._STRIP_KEYS]
        data = ET.tostring(node, pretty_print=True).decode("utf-8")
        return data, self._metadata

In [14]:
d = RichDiagram(drawioUrlParams=dict(ui="dark"))
did = D.display(d, display_id=True)

```md
<mxfile version="13.6.2" etag="sRvBxgpFvqvJ6ZOE7QqG">
  <diagram id="Kgp7oERnrECbv33cBmmy" name="Page-1">
    <mxGraphModel dx="1841" dy="728" grid="1" gridSize="10" guides="1" tooltips="1" connect="1" arrows="1" fold="1" page="1" pageScale="1" pageWidth="850" pageHeight="1100" math="0" shadow="0">
      <root>
        <mxCell id="0"/>
        <mxCell id="1" parent="0"/>
        <mxCell id="3" value="A" style="ellipse;whiteSpace=wrap;html=1;fontSize=48;fontFamily=sans-serif;fillColor=#dae8fc;strokeColor=#6c8ebf;fontColor=#496082;" vertex="1" parent="1">
          <mxGeometry x="20" y="20" width="285" height="280" as="geometry"/>
        </mxCell>
      </root>
    </mxGraphModel>
  </diagram>
</mxfile>

```

In [ ]:
@W.interact
def foo(
    label="B", 
    fontSize=(16, 128),
    x=(-400, 400),
    y=(-400, 400),
    width=(0, 400),
    height=(0, 400)
):
    new = RichDiagram(
        str(d)
        .replace('''value="A"''', f'''value="{label}"''')
        .replace('''fontSize=48''', f'''fontSize={fontSize}pt''')
        .replace('''x="20"''', f'''x="{x}"''')
        .replace('''y="20"''', f'''y="{y}"''')
        .replace('''width="285"''', f'''width="{width}"''')
        .replace('''height="280"''', f'''height="{height}"''')
    )
    did.update({MIME_TYPE: str(new)}, raw=True)